This notebook contains the analysis for Figures 1, 3 and Supplementary figures 1, 2, 3, 4 in the publication ["The Slowing Rate of CpG Depletion in SARS-CoV-2 Genomes Is Consistent with Adaptations to the Human Host"](https://academic.oup.com/mbe/article/39/3/msac029/6521032)

In [1]:
# %reset

# Library imports

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

Library version used:
- scipy: v1.7.1
- np: v1.21.2
- pd: v1.3.3
- sns: v0.11.2
- matplotlib: v3.4.2

# Data import

In [3]:
df_alpha = pd.read_csv('../csv_files/voc_alpha.csv').drop_duplicates()
df_beta = pd.read_csv('../csv_files/voc_beta.csv').drop_duplicates()
df_gamma = pd.read_csv('../csv_files/voc_gamma.csv').drop_duplicates()
df_delta = pd.read_csv('../csv_files/voc_delta.csv').drop_duplicates()
df = pd.read_csv('../csv_files/sars_cov2.csv').drop_duplicates()
df_omicron = pd.read_csv('../csv_files/voc_omicron.csv').drop_duplicates()
absent_accession_ids_in_msa = pd.read_csv('../csv_files/absent_accession_ids_in_msa.csv').drop_duplicates()

print(
    f'df_alpha shape: {df_alpha.shape}\n'
    f'df_beta shape: {df_beta.shape}\n'
    f'df_gamma shape: {df_gamma.shape}\n'
    f'df_delta shape: {df_delta.shape}\n'
    f'df shape: {df.shape}\n'
    f'df_omicron shape: {df_omicron.shape}'
)
print(f'absent_accession_ids_in_msa shape: {absent_accession_ids_in_msa.shape}')

df_alpha shape: (778419, 13)
df_beta shape: (20534, 13)
df_gamma shape: (54138, 13)
df_delta shape: (442530, 13)
df shape: (2151699, 13)
df_omicron shape: (5877, 13)
absent_accession_ids_in_msa shape: (3386, 1)


# Some Initializations

In [4]:
df_alpha.name = 'VOC_alpha'
df_beta.name = 'VOC_beta'
df_gamma.name = 'VOC_gamma'
df_delta.name = 'VOC_delta'
df.name = 'Complete'
df_omicron.name = 'VOC Omicron'

In [5]:
list_of_dfs = [df_alpha, df_beta, df_gamma, df_delta, df, df_omicron]

In [6]:
def get_all_dfs_info():
    print(
        f'df_alpha shape: {df_alpha.shape}\n'
        f'df_beta shape: {df_beta.shape}\n'
        f'df_gamma shape: {df_gamma.shape}\n'
        f'df_delta shape: {df_delta.shape}\n'
        f'df shape: {df.shape}\n'
        f'df_omicron shape: {df_omicron.shape}'
    )

# Extract dates of sample collection, location of sample collection, accession IDs of sequence

In [7]:
for DF in list_of_dfs:
    DF['dates'] = DF['info'].str.rsplit('|', 1).str[1]
    DF['location'] = DF['info'].str.split('/').str[1]
    DF['accession_ids'] = DF['info'].str.split('|').str[1]

get_all_dfs_info()

df_alpha shape: (778419, 16)
df_beta shape: (20534, 16)
df_gamma shape: (54138, 16)
df_delta shape: (442530, 16)
df shape: (2151699, 16)
df_omicron shape: (5877, 16)


# Datetime manipulation, Infection Timeline

In [8]:
def date_manipulations(DF):
    DF.loc[:, 'dates'] = pd.to_datetime(DF.loc[:, 'dates'])
    if not DF.name == 'VOC Omicron':
        DF = DF.loc[DF.loc[:, 'dates'] <= '2021-05-31']
    DF = DF.assign(Infection_Timeline=pd.to_datetime(DF.loc[:, 'dates'], format='%Y-%m-%d'))
    DF.loc[:, 'Infection_Timeline'] = DF.loc[:, 'Infection_Timeline'].dt.strftime('%b-%y')
    DF = DF.sort_values(by='dates')
    return DF


df_alpha = date_manipulations(df_alpha)
df_beta = date_manipulations(df_beta)
df_gamma = date_manipulations(df_gamma)
df_delta = date_manipulations(df_delta)
df = date_manipulations(df)
df_omicron = date_manipulations(df_omicron)

get_all_dfs_info()

df_alpha shape: (720376, 17)
df_beta shape: (18428, 17)
df_gamma shape: (39550, 17)
df_delta shape: (37160, 17)
df shape: (1612859, 17)
df_omicron shape: (5877, 17)


# Data filteration

## Excluding sequences less than 29,700 or more than 30,000 nucleotides

In [9]:
def seq_len_filter(DF):
    DF.query('seq_len >= 29700 and seq_len_adjusted <= 30000', inplace=True)
    # should've used seq_len length only at both places, 
    # makes a diff of 8 entries finally, so, a minor error
    # DF.query('seq_len >= 29700 and seq_len <= 30000', inplace=True)

    
seq_len_filter(df_alpha)
seq_len_filter(df_beta)
seq_len_filter(df_gamma)
seq_len_filter(df_delta)
seq_len_filter(df)
seq_len_filter(df_omicron)

get_all_dfs_info()

df_alpha shape: (663321, 17)
df_beta shape: (16859, 17)
df_gamma shape: (36302, 17)
df_delta shape: (35886, 17)
df shape: (1510286, 17)
df_omicron shape: (5698, 17)


## Excluding sequences with ≥300 Ns (ambiguous base-positions)

In [10]:
def ambiguous_bases_filter(DF):
    DF.query('ambiguous_base_count < 300', inplace=True)


ambiguous_bases_filter(df_alpha)
ambiguous_bases_filter(df_beta)
ambiguous_bases_filter(df_gamma)
ambiguous_bases_filter(df_delta)
ambiguous_bases_filter(df)
ambiguous_bases_filter(df_omicron)

get_all_dfs_info()

df_alpha shape: (662875, 17)
df_beta shape: (16854, 17)
df_gamma shape: (36290, 17)
df_delta shape: (35855, 17)
df shape: (1508737, 17)
df_omicron shape: (5693, 17)


## Separating unclassified from variants

NOTE: There was a difference (of around a week) in date of sequence collection for the complete dataset and for the variants from GISAID. The difference of 52 entries in the following code-block pertains to that.

In [11]:
df_all_var = pd.concat([df_alpha, df_beta, df_gamma, df_delta])
print(f'df_all_var shape: {df_all_var.shape}')

df_unclassified = df[~df.accession_ids.isin(df_all_var.accession_ids)]
print(f'df_unclassified shape: {df_unclassified.shape}')

print(f'Difference: {len(df) - len(df_all_var) - len(df_unclassified)}')

df_all_var shape: (751874, 17)
df_unclassified shape: (756915, 17)
Difference: -52


In [12]:
df_all_var.name = 'All_variants'
df_unclassified.name = 'Unclassified'

# Redefining to add df_unclassified in our quick print method
def get_all_dfs_info():
    print(
        f'df_alpha shape: {df_alpha.shape}\n'
        f'df_beta shape: {df_beta.shape}\n'
        f'df_gamma shape: {df_gamma.shape}\n'
        f'df_delta shape: {df_delta.shape}\n'
        f'df_unclassified shape: {df_unclassified.shape}\n'
        f'df shape: {df.shape}\n'
        f'df_omicron shape: {df_omicron.shape}'
    )

## Excluding sequences with a Z score ≥3 for nucleotide composition, GC content, dinucleotide odds ratio for CpG and GpC, and CpG numbers and percentages

In [13]:
def zscore_filtration(DF, colname_list):
    for colname in colname_list:
        DF = DF[np.abs(stats.zscore(DF[colname])) < 3]
    return DF

In [14]:
list_of_cols = [
    'count_CG', 
    'ObyE_CG', 
    'ObyE_GC', 
    'percent_CG', 
    'percent_A', 
    'percent_C', 
    'percent_G', 
    'percent_T', 
    'GC_content'
]

df_alpha = zscore_filtration(df_alpha, list_of_cols)
df_beta = zscore_filtration(df_beta, list_of_cols)
df_gamma = zscore_filtration(df_gamma, list_of_cols)
df_delta = zscore_filtration(df_delta, list_of_cols)
df_unclassified = zscore_filtration(df_unclassified, list_of_cols)
df = zscore_filtration(df, list_of_cols)
df_omicron = zscore_filtration(df_omicron, list_of_cols)

get_all_dfs_info()

df_alpha shape: (618066, 17)
df_beta shape: (16286, 17)
df_gamma shape: (34300, 17)
df_delta shape: (32121, 17)
df_unclassified shape: (719601, 17)
df shape: (1413725, 17)
df_omicron shape: (5436, 17)


## Excluding entries from outside China before January 13, 2020, as there were no documented cases outside China in this period (Li et al. 2020)

Such an entry shouln't exist in any of these variants dataset, so, it'll act as a check as well.

In [15]:
def nonwuhan_before_jan13(DF):
    DF.drop(
        DF[
            (DF['dates'] < "2020-01-13") & (
                (DF['location'] == 'USA')
                | (DF['location'] == 'Japan')
                | (DF['location'] == 'Thailand')
            )

        ].index,
        inplace=True
    )
    return DF


df_alpha = nonwuhan_before_jan13(df_alpha)
df_beta = nonwuhan_before_jan13(df_beta)
df_gamma = nonwuhan_before_jan13(df_gamma)
df_delta = nonwuhan_before_jan13(df_delta)
df_unclassified = nonwuhan_before_jan13(df_unclassified)
df = nonwuhan_before_jan13(df)
df_omicron = nonwuhan_before_jan13(df_omicron)
    
get_all_dfs_info()

df_alpha shape: (618066, 17)
df_beta shape: (16286, 17)
df_gamma shape: (34300, 17)
df_delta shape: (32121, 17)
df_unclassified shape: (719588, 17)
df shape: (1413714, 17)
df_omicron shape: (5436, 17)


## Excluding the full-length sequences that were not included in the multiple sequence alignment (MSA) available in GISAID (as on September 6, 2021 for samples collected from January 1, 2020 to May 31, 2021)

In [16]:
def seq_na_in_msa_filter(DF):
    DF = DF[~DF.accession_ids.isin(absent_accession_ids_in_msa.accession_ids)]
    return DF


df_alpha = seq_na_in_msa_filter(df_alpha)
df_beta = seq_na_in_msa_filter(df_beta)
df_gamma = seq_na_in_msa_filter(df_gamma)
df_delta = seq_na_in_msa_filter(df_delta)
df_unclassified = seq_na_in_msa_filter(df_unclassified)
df = seq_na_in_msa_filter(df)
df_omicron = seq_na_in_msa_filter(df_omicron)

get_all_dfs_info()

df_alpha shape: (616626, 17)
df_beta shape: (16247, 17)
df_gamma shape: (34162, 17)
df_delta shape: (32109, 17)
df_unclassified shape: (717950, 17)
df shape: (1410423, 17)
df_omicron shape: (5436, 17)


# Location-wise stats

In [17]:
def location_stats(DF):
    print(f'\nFor {DF.name}, Top 10 locations on the basis of frequency of sequence submissions in our data: \n\nPercentages: ')

    print(DF['location'].value_counts(normalize = True).mul(100).head(10))
    print('\nCounts: ')
    print(DF['location'].value_counts().sort_values(ascending = False).head(10))


# I have given names to the dataframes above but for some reason it 
# keeps throwing error at random runs, so, being defensive about it.
try:
    location_stats(df_alpha)
    location_stats(df_beta)
    location_stats(df_gamma)
    location_stats(df_delta)
    location_stats(df_unclassified)
    location_stats(df)
    location_stats(df_omicron)
except AttributeError:
    df_alpha.name = 'VOC_alpha'
    df_beta.name = 'VOC_beta'
    df_gamma.name = 'VOC_gamma'
    df_delta.name = 'VOC_delta'
    df.name = 'Complete'
    df_all_var.name = 'All_variants'
    df_unclassified.name = 'Unclassified'
    location_stats(df_alpha)
    location_stats(df_beta)
    location_stats(df_gamma)
    location_stats(df_delta)
    location_stats(df_unclassified)
    location_stats(df)
    location_stats(df_omicron)



For VOC_alpha, Top 10 locations on the basis of frequency of sequence submissions in our data: 

Percentages: 
England        25.661260
USA            16.555092
Germany         9.650420
Denmark         7.407407
Sweden          5.912174
Japan           4.350125
Scotland        3.507312
Netherlands     3.298596
France          2.708287
Switzerland     2.077272
Name: location, dtype: float64

Counts: 
England        158234
USA            102083
Germany         59507
Denmark         45676
Sweden          36456
Japan           26824
Scotland        21627
Netherlands     20340
France          16700
Switzerland     12809
Name: location, dtype: int64

For VOC_beta, Top 10 locations on the basis of frequency of sequence submissions in our data: 

Percentages: 
South_Africa    12.980858
Sweden          10.389610
France          10.014156
USA              7.225949
Germany          6.973595
Finland          5.816458
Philippines      4.825506
Belgium          4.394657
Reunion          4.136148
Net

AttributeError: 'DataFrame' object has no attribute 'name'

# Saving the dataframes and supplementary tables

## Saving Supplementary Tables

### Supplementary Table 1

In [31]:
first_mil = df['accession_ids'].iloc[:1000000]
first_mil.reset_index(inplace=True, drop=True)
after_mil = df['accession_ids'].iloc[1000000:]
after_mil.reset_index(inplace=True, drop=True)

accession_ids = pd.concat(
    [first_mil, after_mil], 
    axis=1, 
    ignore_index=True
)

accession_ids.rename(
    columns={
        0: 'Accession ids | (1-1000000)', 
        1: 'Accession ids | (1000001-1410423)'
    }, 
    inplace=True
)

# accession_ids.to_excel(
#     '../supplementary_tables/Supplementary Table 1.xlsx', 
#     index=False, 
#     sheet_name='Acknowledgement Table', 
#     header=[
#         'Accession ids | (1-1000000)', 
#         'Accession ids | (1000001-1410423)'
#     ]
# )

accession_ids.head()

,Accession ids | (1-1000000),Accession ids | (1000001-1410423)
0,EPI_ISL_402120,EPI_ISL_2860172
1,EPI_ISL_403928,EPI_ISL_2860168
2,EPI_ISL_406800,EPI_ISL_2860164
3,EPI_ISL_406717,EPI_ISL_2860155
4,EPI_ISL_406716,EPI_ISL_2860149


### Supplementary Table 3

In [21]:
df_alpha.reset_index(drop=True, inplace=True)
df_beta.reset_index(drop=True, inplace=True)
df_gamma.reset_index(drop=True, inplace=True)
df_delta.reset_index(drop=True, inplace=True)
df_unclassified.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df_omicron.reset_index(drop=True, inplace=True)

In [27]:
supp_3 = pd.concat([df_alpha.accession_ids, df_beta.accession_ids, df_gamma.accession_ids,
          df_delta.accession_ids, df_unclassified.accession_ids], axis=1)

supp_3.columns.values[0]='VOC Alpha'
supp_3.columns.values[1]='VOC Beta'
supp_3.columns.values[2]='VOC Gamma'
supp_3.columns.values[3]='VOC Delta'
supp_3.columns.values[4]='Non-VOCs'

## Saving to csv
# supp_3.to_csv('../supplementary_tables/Supplementary Table 3.csv', index=False)

supp_3.head()

,VOC Alpha,VOC Beta,VOC Gamma,VOC Delta,Non-VOCs
0,EPI_ISL_2898105,EPI_ISL_2508437,EPI_ISL_2612488,EPI_ISL_2937904,EPI_ISL_402120
1,EPI_ISL_2103488,EPI_ISL_1191821,EPI_ISL_540908,EPI_ISL_1663516,EPI_ISL_403928
2,EPI_ISL_2282167,EPI_ISL_2802129,EPI_ISL_577484,EPI_ISL_3263174,EPI_ISL_406800
3,EPI_ISL_2374676,EPI_ISL_2140654,EPI_ISL_573874,EPI_ISL_3132911,EPI_ISL_406717
4,EPI_ISL_2279938,EPI_ISL_700551,EPI_ISL_726669,EPI_ISL_3132910,EPI_ISL_406716


### Supplementary Table 7

In [26]:
supp_7 = df_omicron.loc[:, ['accession_ids']]
supp_7.columns.values[0]='VOC Omicron'

# supp_7.to_csv('../supplementary_tables/Supplementary Table 7.csv', index=False)

supp_7.head()

,VOC Omicron
0,EPI_ISL_7337441
1,EPI_ISL_7337502
2,EPI_ISL_8604930
3,EPI_ISL_7543747
4,EPI_ISL_8604928


## Saving dataframes

In [ ]:
# df_alpha.to_csv('../csv_files/filtered_voc_alpha.csv', index=False)
# df_beta.to_csv('../csv_files/filtered_voc_beta.csv', index=False)
# df_gamma.to_csv('../csv_files/filtered_voc_gamma.csv', index=False)
# df_delta.to_csv('../csv_files/filtered_voc_delta.csv', index=False)
# df_unclassified.to_csv('../csv_files/filtered_non_vocs.csv', index=False)
# df.to_csv('../csv_files/filtered_sars_cov2.csv', index=False)
# df_omicron.to_csv('../csv_files/filtered_voc_omicron.csv', index=False)

In [ ]:
# df[~df.accession_ids.isin(df_gamma.accession_ids)].loc[
#     :, ['Infection_Timeline', 'count_CG']
# ].to_csv(
#     '../csv_files/nongamma_count_CG.csv', index=False
# )